In [1]:
import numpy as np
import itertools

# ------------------------
# Parameters
# ------------------------
degree_theta = 1.25  # Example twist angle (deg)

hbar = 0.6582  # (eV*fs)
m0 = 5.68568  # (eV fs² nm⁻²)

me = 0.43 * m0
phi = 128 * (np.pi / 180)
V0 = 0.009
w = 0.018
a = 0.3317
theta = degree_theta * (np.pi / 180)

# Moiré lattice constant
aM = a / (2.0 * np.sin(0.5 * theta))

# g-vectors of mBZ
g0 = (4 * np.pi) / (np.sqrt(3) * aM)
q0 = g0 / np.sqrt(3)
gvecs = g0 * np.array([[np.cos(n * np.pi / 3), np.sin(n * np.pi / 3)] for n in range(6)])
Ktop = (gvecs[0] + gvecs[5]) / 3
Kbot = (gvecs[0] + gvecs[1]) / 3
g1 = gvecs[0]
g2 = gvecs[1]

# Moiré shells
Nshells = 4
Nzf = (2 * Nshells + 1) ** 2
map_zf_idx = list(itertools.product(range(-Nshells, Nshells + 1),
                                    range(-Nshells, Nshells + 1)))
Nbands = Nzf * 4  # 2 layers * 2 sublattices

# ------------------------
# Functions (unchanged logic)
# ------------------------
def compute_H(k):
    H = np.zeros((Nbands, Nbands), dtype=complex)

    for n1 in range(Nzf):
        n11, n12 = map_zf_idx[n1]
        kmx_t, kmy_t = k + n11 * g1 + n12 * g2 - Ktop
        kmx_b, kmy_b = k + n11 * g1 + n12 * g2 - Kbot

        H[n1*4, n1*4]     = (hbar**2) * (kmx_t**2 + kmy_t**2) / (2 * me)
        H[n1*4+1, n1*4+1] = (hbar**2) * (kmx_t**2 + kmy_t**2) / (2 * me)
        H[n1*4+2, n1*4+2] = (hbar**2) * (kmx_b**2 + kmy_b**2) / (2 * me)
        H[n1*4+3, n1*4+3] = (hbar**2) * (kmx_b**2 + kmy_b**2) / (2 * me)

        H[n1*4, n1*4+2]   = w
        H[n1*4+2, n1*4]   = w
        H[n1*4+1, n1*4+3] = w
        H[n1*4+3, n1*4+1] = w

        for n2 in range(Nzf):
            n21, n22 = map_zf_idx[n2]
            if (n21 == n11 + 1) and (n22 == n12):
                H[n1*4, n2*4]     = -V0 * np.exp(1j * phi)
                H[n1*4+1, n2*4+1] = -V0 * np.exp(1j * phi)
                H[n1*4+2, n2*4+2] = -V0 * np.exp(-1j * phi)
                H[n1*4+3, n2*4+3] = -V0 * np.exp(-1j * phi)

                H[n2*4, n1*4]     = -V0 * np.exp(-1j * phi)
                H[n2*4+1, n1*4+1] = -V0 * np.exp(-1j * phi)
                H[n2*4+2, n1*4+2] = -V0 * np.exp(1j * phi)
                H[n2*4+3, n1*4+3] = -V0 * np.exp(1j * phi)
            elif (n21 == n11) and (n22 == n12 + 1):
                H[n1*4, n2*4]     = -V0 * np.exp(-1j * phi)
                H[n1*4+1, n2*4+1] = -V0 * np.exp(-1j * phi)
                H[n1*4+2, n2*4+2] = -V0 * np.exp(1j * phi)
                H[n1*4+3, n2*4+3] = -V0 * np.exp(1j * phi)
                H[n1*4, n2*4+2]   = w
                H[n1*4+1, n2*4+3] = w

                H[n2*4, n1*4]     = -V0 * np.exp(1j * phi)
                H[n2*4+1, n1*4+1] = -V0 * np.exp(1j * phi)
                H[n2*4+2, n1*4+2] = -V0 * np.exp(-1j * phi)
                H[n2*4+3, n1*4+3] = -V0 * np.exp(-1j * phi)
                H[n2*4+2, n1*4]   = w
                H[n2*4+3, n1*4+1] = w
            elif (n21 == n11 - 1) and (n22 == n12 + 1):
                H[n1*4, n2*4]     = -V0 * np.exp(1j * phi)
                H[n1*4+1, n2*4+1] = -V0 * np.exp(1j * phi)
                H[n1*4+2, n2*4+2] = -V0 * np.exp(-1j * phi)
                H[n1*4+3, n2*4+3] = -V0 * np.exp(-1j * phi)
                H[n1*4, n2*4+2]   = w
                H[n1*4+1, n2*4+3] = w

                H[n2*4, n1*4]     = -V0 * np.exp(-1j * phi)
                H[n2*4+1, n1*4+1] = -V0 * np.exp(-1j * phi)
                H[n2*4+2, n1*4+2] = -V0 * np.exp(1j * phi)
                H[n2*4+3, n1*4+3] = -V0 * np.exp(1j * phi)
                H[n2*4+2, n1*4]   = w
                H[n2*4+3, n1*4+1] = w

    return H

def solve_H(k):
    H = compute_H(k)
    eigenvalues, eigenvectors = np.linalg.eigh(H)
    Ek = np.real(eigenvalues)
    WF = eigenvectors
    return Ek, WF

def calculate_bandstructure():
    Nk1 = 100
    Nk2 = 50
    Nk3 = 100
    Nk = Nk1 + Nk2 + Nk3
    
    kvec = []
    Gamma = np.array([0, 0])
    for n in range(Nk1):
        kvec.append(Gamma + (Ktop - Gamma) * (n / Nk1))
    Mpoint = 0.5 * (Ktop + Kbot)
    for n in range(Nk2):
        kvec.append(Ktop + (Mpoint - Ktop) * (n / Nk2))
    for n in range(Nk3):
        kvec.append(Mpoint + (Gamma - Mpoint) * (n / Nk3))

    kvec = np.array(kvec)

    Ek = np.zeros((Nbands, Nk))
    WF = None
    for nk in range(Nk):
        Ek[:, nk], WF = solve_H(kvec[nk, :]) #Structure of this is that index 0 is the band index and the rest are the momentum positioning

    bz_points = [0, Nk1, Nk1 + Nk2, Nk]

    return Ek, WF, bz_points, Nk

Ek, WF, bz_points, Nk = calculate_bandstructure()

In [ ]:
Nkx = 41
Nky = int(round(Nkx * 2.0 / np.sqrt(3)))
kxvec = np.linspace(0.0, 1.0, Nkx) * g0
kyvec = np.linspace(-1, 0.5, Nky) * q0
dkx = kxvec[1] - kxvec[0]
dky = kyvec[1] - kyvec[0]

band = 0
#E = np.zeros((Nkx, Nky))
WF_iter = np.zeros((Nbands,Nkx))
WF = np.zeros((Nbands, Nkx, Nky), dtype=complex)
for nkx in range(Nkx):
    for nky in range(Nky):
        k = np.array([kxvec[nkx], kyvec[nky]])
        _, WF_hlp = solve_H(k) #Calculating the energy for all bands
        WF_iter[:,nkx] = WF_hlp[:, band]
        #E[nkx, nky] = E_hlp[band] # Saving the value for the selected band in the new energy vector
        WF[:, nkx, nky] = WF_hlp[:, band] # Saving the eigenvector for the selected band in the new energy vector

Berry_curvature = np.zeros((Nkx-1, Nky-1))
Berry_curvature_1 = np.zeros((Nkx-1, Nky-1))
for nkx in range(Nkx-1):
    for nky in range(Nky-1): # Eq 16 in the article
        
        Ux = np.sum(np.conj(WF[:, nkx, nky]) * WF[:, nkx+1, nky])
        Uy = np.sum(np.conj(WF[:, nkx, nky]) * WF[:, nkx, nky+1])
        Uxy = np.sum(np.conj(WF[:, nkx+1, nky]) * WF[:, nkx+1, nky+1])
        Uyx = np.sum(np.conj(WF[:, nkx, nky+1]) * WF[:, nkx+1, nky+1])

        # Wilson loop
        Wloop = Ux * Uxy * np.conj(Uyx * Uy)
                
        # Ensure that the phase is between -pi and +pi
        phase = np.angle(Wloop)
        phase = np.mod(phase + np.pi, 2*np.pi) - np.pi
                
        Berry_curvature[nkx, nky] = phase / (dkx * dky)


chernnum = np.sum(Berry_curvature) * dkx * dky / (2 * np.pi)

        # U12  = np.sum(np.conj(WF[:, nkx, nky])   * WF[:, nkx+1, nky]) 
        # U23 = np.sum(np.conj(WF[:, nkx+1, nky]) * WF[:, nkx+1, nky+1]) # ??? I am not sure
        # U34 = np.sum(np.conj(WF[:, nkx+1, nky+1] * WF[:, nkx, nky+1]))
        # U41  = np.sum(np.conj(WF[:, nkx, nky+1] * WF[:, nkx, nky]))
        # #TODO Look thtought equations 8-16 and try to look at what s going on
        # # https://www.openmx-square.org/openmx_man3.9/node184.html

        # F += np.angle(np.log(U12*U23*np.conj(U34)*np.conj(U41)))
        # phase += np.mod(F + np.pi, 2*np.pi) - np.pi
        # Berry_curvature_1[nkx, nky] = phase / (dkx * dky)
        
# chernnum1 = (1/(2*np.pi))*np.sum(Berry_curvature_1)
# chernnum2 = np.sum(Berry_curvature) * dkx * dky / (2 * np.pi)

print(f"Chern number: {chernnum}")


/tmp/ipykernel_423/2081101170.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  WF_iter[:,nkx] = WF_hlp[:, band]


Chern number: 15.655182724662756


In [58]:
# Berry curvature over momentum, color map
k = np.zeros((Nkx,Nky,2))
for nkx in range(Nkx):
    for nky in range(Nkx):
        #k = [kxvec[nkx],kyvec[nky]]
        k[nkx,nky] = (nkx-1)/(Nkx-1)*g1 + (nky-1)/(Nky-1)*g2

k

array([[[-7.15770494e-02, -4.13250288e-02],
        [-4.77180330e-02,  0.00000000e+00],
        [-2.38590165e-02,  4.13250288e-02],
        [ 1.38777878e-17,  8.26500575e-02],
        [ 2.38590165e-02,  1.23975086e-01],
        [ 4.77180330e-02,  1.65300115e-01],
        [ 7.15770494e-02,  2.06625144e-01],
        [ 9.54360659e-02,  2.47950173e-01],
        [ 1.19295082e-01,  2.89275201e-01],
        [ 1.43154099e-01,  3.30600230e-01],
        [ 1.67013115e-01,  3.71925259e-01]],

       [[-2.38590165e-02, -4.13250288e-02],
        [ 0.00000000e+00,  0.00000000e+00],
        [ 2.38590165e-02,  4.13250288e-02],
        [ 4.77180330e-02,  8.26500575e-02],
        [ 7.15770494e-02,  1.23975086e-01],
        [ 9.54360659e-02,  1.65300115e-01],
        [ 1.19295082e-01,  2.06625144e-01],
        [ 1.43154099e-01,  2.47950173e-01],
        [ 1.67013115e-01,  2.89275201e-01],
        [ 1.90872132e-01,  3.30600230e-01],
        [ 2.14731148e-01,  3.71925259e-01]],

       [[ 2.38590165e-02, -4

In [61]:
kxvec = np.linspace(g1[0], g2[0], Nkx)
kyvec = np.linspace(g1[1], g2[1], Nky)
dkx = kxvec[1] - kxvec[0]
dky = kyvec[1] - kyvec[0]
k = [kxvec,kyvec]
k

[array([0.47718033, 0.45332131, 0.4294623 , 0.40560328, 0.38174426,
        0.35788525, 0.33402623, 0.31016721, 0.2863082 , 0.26244918,
        0.23859016]),
 array([0.        , 0.04132503, 0.08265006, 0.12397509, 0.16530012,
        0.20662514, 0.24795017, 0.2892752 , 0.33060023, 0.37192526,
        0.41325029])]

In [ ]:
# Define k-grid in mBZ (map hexagon into square)
Nkx = 11 # Number of points in x direction
Nky = 11
band = 2

# kxvec = np.linspace(g1[0], g2[0], Nkx)
# kyvec = np.linspace(g1[1], g2[1], Nky)
# dkx = kxvec[1] - kxvec[0]
# dky = kyvec[1] - kyvec[0]

WF = np.zeros((Nbands, Nkx, Nky), dtype=complex)
for nkx in range(Nkx):
    for nky in range(Nkx):
        #k = [kxvec[nkx],kyvec[nky]]
        k = ((nkx-1)/(Nkx-1))*g1 + ((nky-1)/(Nky-1))*g2
        _, WF_hlp = solve_H(k)
        #E[:,nkx,nky] = real.([F.values[band], F.values[band+1]])
        WF[:,nkx,nky] = WF_hlp[:,band]


Berry_curvature = np.zeros((Nkx - 1, Nky - 1))
for nkx in range(Nkx - 1):
    for nky in range(Nky - 1):
        Ux  = np.vdot(WF[:, nkx, nky], WF[:, nkx + 1, nky])      # <ψ(k)|ψ(k+dx)>
        Uy  = np.vdot(WF[:, nkx, nky], WF[:, nkx, nky + 1])      # <ψ(k)|ψ(k+dy)>
        Uxy = np.vdot(WF[:, nkx + 1, nky], WF[:, nkx + 1, nky + 1])
        Uyx = np.vdot(WF[:, nkx, nky + 1], WF[:, nkx + 1, nky + 1])
        
        # Ensure phase ∈ [-π, π]
        phase = np.angle(Ux * Uxy * np.conj(Uyx * Uy))
        phase = np.mod(phase + np.pi, 2 * np.pi) - np.pi

        Berry_curvature[nkx, nky] = phase #/ (dkx * dky) #if you want normalization

chernnum = np.sum(Berry_curvature) / (2 * np.pi) #* dkx * dky 

print(f"Chern number: {chernnum}")


Chern number: -2.8202300601598593


In [ ]:
#Nkx = 11, band = 0: -0.0014881851122035999
#Nkx = 21, band = 0: 0.0006489733799771645

In [34]:
def compute_Chern_number(band):
    Nkx = 21
    Nky = int(round(Nkx * 2.0 / np.sqrt(3)))
    kxvec = np.linspace(0.0, 1.0, Nkx) * g0
    kyvec = np.linspace(-1, 0.5, Nky) * q0
    dkx = kxvec[1] - kxvec[0]
    dky = kyvec[1] - kyvec[0]

    WF = np.zeros((Nbands, Nkx, Nky), dtype=complex)
    for nkx in range(Nkx):
        for nky in range(Nky):
            k = np.array([kxvec[nkx], kyvec[nky]])
            _, WF_hlp = solve_H(k)
            WF[:, nkx, nky] = WF_hlp[:, band] # Wave function of the selected band

    Berry_curvature = np.zeros((Nkx-1, Nky-1))
    for nkx in range(Nkx-1):
        for nky in range(Nky-1):
            Ux  = np.sum(np.conj(WF[:, nkx, nky])   * WF[:, nkx+1, nky]) #Eq 16 in the paper
            Uy  = np.sum(np.conj(WF[:, nkx, nky])   * WF[:, nkx, nky+1])

            Uxy = np.sum(np.conj(WF[:, nkx+1, nky]) * WF[:, nkx+1, nky+1])
            Uyx = np.sum(np.conj(WF[:, nkx, nky+1]) * WF[:, nkx+1, nky+1])

            Wloop = Ux * Uxy * np.conj(Uyx * Uy)

            phase = np.angle(Wloop)
            #phase = np.mod(phase + np.pi, 2*np.pi) - np.pi
            Berry_curvature[nkx, nky] = phase / (dkx * dky)

    chernnum = np.sum(Berry_curvature) * dkx * dky / (2 * np.pi)
    print(f"Chern number: {chernnum}")

    return chernnum